In [1]:
"""This file is a test playground for new scripts and functions."""

%load_ext autoreload
%autoreload 2

###### USER SETTINGS ######
SERIAL_ACTIVE = False # if False, just runs the algorithm without sending to HV switches

###### INITIALIZATIONS ######
import cv2
import time
import serial
import numpy as np
import visual_haptic_utils.haptic_funcs as haptic_funcs # my custom file
import visual_haptic_utils.USB_writer as USB_writer # my custom file
import matplotlib.pyplot as plt

###### MAIN ######

# initialize camera and gesture model:
frame_rate = 24
generator = haptic_funcs.IntensityGenerator(total_time=5, frame_rate=frame_rate)
haptic_map = haptic_funcs.HapticMap()

output_list = []
output_list.extend(generator.ramp(direction=1))
output_list.extend(generator.ramp(direction=-1))
output_list.extend(generator.sine_global(freq=1))
output_list.extend(generator.checker_sine(freq=0.5))
output_list.extend(generator.checker_square(freq=0.5))

duty_array_list, period_array_list = haptic_map.linear_map(output_list,
                                                           freq_range=(0,200),
                                                           duty_range=(0.05,0.5))


In [9]:
duty_array_list[300]

array([[0.275, 0.275, 0.275, 0.275, 0.275, 0.275, 0.275],
       [0.275, 0.275, 0.275, 0.275, 0.275, 0.275, 0.275],
       [0.275, 0.275, 0.275, 0.275, 0.275, 0.275, 0.275],
       [0.275, 0.275, 0.275, 0.275, 0.275, 0.275, 0.275]])